# SQL Server Database

In [1]:
import sqlalchemy
from sqlalchemy import text, bindparam
from functools import partial
import pandas as pd
from sql_connector import DB
import re
import hashlib
import numpy as np

# show all columns
pd.set_option('display.max_columns', None)

Connection to NBA was successful.


## Create Engine

In [2]:
nba = DB(db_name='NBA')
tempdb = DB(db_name='tempdb')

### Disable and Enable FK Constrains, IDENTITY_INSERT to Insert Data

#### Read Data

In [326]:
player_shotLocations = pd.read_csv(r"D:\lianz\Desktop\Python\personal_projects\nba_airflow\data\player_shotLocations.csv", skipinitialspace=True, index_col=0)
games_played = pd.read_csv(r"D:\lianz\Desktop\Python\personal_projects\nba_airflow\data\games_played.csv", parse_dates=['DateTime'])
players = pd.read_csv(r'D:\lianz\Desktop\Python\personal_projects\nba_airflow\data\players.csv', parse_dates=['BirthDate'], names=['PlayerName', 'FromYear', 'ToYear', 'PositionID', 'Height', 'Weight', 'BirthDate', 'College'], header=0)
play_by_play = pd.read_parquet(r"D:\lianz\Desktop\Python\personal_projects\nba_airflow\data\play_by_play.parquet")

#### Insert Positions

In [62]:
positions = pd.read_csv(r'D:\lianz\Desktop\Python\personal_projects\nba_airflow\data\position.csv')
positions

,PositionID,PositionName,Description
0,1,F-C,Forward-center
1,2,C-F,Center-forward
2,3,C,Center
3,4,G,Guard
4,5,F,Forward
5,6,G-F,Guard-forward
6,7,F-G,Forward-guard


In [ ]:
# Disable FK constrains and set IDENTITY_INSERT to ON
with nba.engine.connect() as connection:
    connection.execute(text("EXEC sp_MSforeachtable @command1='ALTER TABLE ? NOCHECK CONSTRAINT ALL'"))
    connection.execute(text("SET IDENTITY_INSERT Position ON"))

# Insert positions table
positions.to_sql('Position', con=nba.engine, if_exists='append', index=False)  # Use the engine here

# Re-enable FK constraints and set IDENTITY_INSERT to OFF
with nba.engine.connect() as connection:
    connection.execute(text("SET IDENTITY_INSERT Position OFF"))
    connection.execute(text("EXEC sp_MSforeachtable @command1='ALTER TABLE ? WITH CHECK CHECK CONSTRAINT ALL'"))


#### Insert Players

In [ ]:
players = pd.read_csv(r'D:\lianz\Desktop\Python\personal_projects\nba_airflow\data\players.csv', parse_dates=['BirthDate'], names=['PlayerName', 'FromYear', 'ToYear', 'PositionID', 'Height', 'Weight', 'BirthDate', 'College'], header=0)

# Read positions from sql
with nba.engine.connect() as connection:
    positions = connection.execute(text("SELECT * FROM Position")).fetchall()
    positions = pd.DataFrame(positions, columns=['PositionID', 'PositionName', 'Description'])

# Join positions to players
players = players.merge(positions, left_on='PositionID', right_on='PositionName', how='left', suffixes=('_x', ''))\
    .drop(columns=['PositionID_x', 'PositionName', 'Description'])

In [ ]:
# Disable FK constrains and set IDENTITY_INSERT to ON
with nba.engine.connect() as connection:
    connection.execute(text("EXEC sp_MSforeachtable @command1='ALTER TABLE ? NOCHECK CONSTRAINT ALL'"))
    # connection.execute(text("SET IDENTITY_INSERT Player ON"))

# Insert positions table
players.to_sql('Player', con=nba.engine, if_exists='append', index=False)  # Use the engine here

# Re-enable FK constraints and set IDENTITY_INSERT to OFF
with nba.engine.connect() as connection:
    # connection.execute(text("SET IDENTITY_INSERT Player OFF"))
    connection.execute(text("EXEC sp_MSforeachtable @command1='ALTER TABLE ? WITH CHECK CHECK CONSTRAINT ALL'"))


#### Clean Players (sql)

In [355]:
with nba.engine.connect() as connection:
    players_sql = connection.execute(text('SELECT * FROM Player')).fetchall()
    players_sql = pd.DataFrame(players_sql, columns=['PlayerID', 'PlayerName', 'FromYear', 'ToYear', 'PositionID', 'Height', 'Weight', 'BirthDate', 'College'])

# Find all special characters in PlayerName and convert to simplified alphanumeric value
special_chars = players['PlayerName'].str.findall(r'[^a-zA-Z0-9 ]')
special_chars = [re.sub(r'\W+', '_', char).lower() for sublist in special_chars for char in sublist]
special_chars = list(set(special_chars))
special_chars


2023-08-27 21:27:36,595 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-08-27 21:27:36,596 INFO sqlalchemy.engine.Engine SELECT * FROM Player
2023-08-27 21:27:36,596 INFO sqlalchemy.engine.Engine [cached since 891.7s ago] ()
2023-08-27 21:27:36,643 INFO sqlalchemy.engine.Engine ROLLBACK


['ū',
 '_',
 'ń',
 'i̇',
 'è',
 'ã',
 'ğ',
 'ı',
 'ö',
 'ý',
 'á',
 'ú',
 'ș',
 'ê',
 'ē',
 'ß',
 'ģ',
 'ä',
 'ā',
 'ï',
 'š',
 'ç',
 'ć',
 'í',
 'ü',
 'ò',
 'ô',
 'é',
 'ž',
 'ş',
 'ó',
 'ř',
 'č',
 'ð',
 'ņ',
 'ë']

#### Create and Insert Arena csv

In [ ]:
arena = teams[['Arena', 'Attendance']].groupby('Arena').max().reset_index()
arena.columns = ['ArenaName', 'Capacity']
arena.to_csv(r'D:\lianz\Desktop\Python\personal_projects\nba_airflow\data\arena.csv', index=False)

In [216]:
arena = pd.read_csv(r'D:\lianz\Desktop\Python\personal_projects\nba_airflow\data\arena.csv')
arena

,ArenaName,Capacity
0,ARCO Arena (II),17317.0
1,AT&T Center,19615.0
2,AccorHotels Arena,15885.0
3,Air Canada Centre,21050.0
4,Alamodome,68323.0
...,...,...
88,Visa Athletic Center,NaN
89,Vivint Arena,18206.0
90,Vivint Smart Home Arena,19911.0
91,Wachovia Center,21068.0


In [217]:

# Disable FK constrains and set IDENTITY_INSERT to ON
with nba.engine.connect() as connection:
    connection.execute(text("EXEC sp_MSforeachtable @command1='ALTER TABLE ? NOCHECK CONSTRAINT ALL'"))
    # connection.execute(text("SET IDENTITY_INSERT Player ON"))

# Insert positions table
arena.to_sql('Arena', con=nba.engine, if_exists='append', index=False)  # Use the engine here

# Re-enable FK constraints and set IDENTITY_INSERT to OFF
with nba.engine.connect() as connection:
    # connection.execute(text("SET IDENTITY_INSERT Player OFF"))
    connection.execute(text("EXEC sp_MSforeachtable @command1='ALTER TABLE ? WITH CHECK CHECK CONSTRAINT ALL'"))


2023-08-23 01:09:04,303 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-08-23 01:09:04,304 INFO sqlalchemy.engine.Engine EXEC sp_MSforeachtable @command1='ALTER TABLE ? NOCHECK CONSTRAINT ALL'
2023-08-23 01:09:04,304 INFO sqlalchemy.engine.Engine [cached since 8.826e+04s ago] ()
2023-08-23 01:09:04,356 INFO sqlalchemy.engine.Engine ROLLBACK
2023-08-23 01:09:04,364 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-08-23 01:09:04,365 INFO sqlalchemy.engine.Engine SELECT [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] 
FROM [INFORMATION_SCHEMA].[TABLES] 
WHERE ([INFORMATION_SCHEMA].[TABLES].[TABLE_TYPE] = CAST(? AS NVARCHAR(max)) OR [INFORMATION_SCHEMA].[TABLES].[TABLE_TYPE] = CAST(? AS NVARCHAR(max))) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] = CAST(? AS NVARCHAR(max)) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_SCHEMA] = CAST(? AS NVARCHAR(max))
2023-08-23 01:09:04,365 INFO sqlalchemy.engine.Engine [cached since 8.839e+04s ago] ('BASE TABLE', 'VIEW', 'Arena', 'dbo')
2023-08-23 01:09:

#### Create and Insert Teams csv

In [130]:
# Get unique teams from games_played
games_played[['Home', 'Home_short']].drop_duplicates().reset_index(drop=True).to_csv(r'D:\lianz\Desktop\Python\personal_projects\nba_airflow\data\team.csv', index=False)

In [223]:
teams = pd.read_csv(r'D:\lianz\Desktop\Python\personal_projects\nba_airflow\data\team.csv', names=['TeamName', 'TeamShort'], header=0)
teams.head()

,TeamName,TeamShort
0,Atlanta Hawks,ATL
1,New Jersey Nets,NJN
2,Orlando Magic,ORL
3,Dallas Mavericks,DAL
4,New York Knicks,NYK


In [224]:
# Disable FK constrains and set IDENTITY_INSERT to ON
with nba.engine.connect() as connection:
    connection.execute(text("EXEC sp_MSforeachtable @command1='ALTER TABLE ? NOCHECK CONSTRAINT ALL'"))
    # connection.execute(text("SET IDENTITY_INSERT Player ON"))

# Insert positions table
teams.to_sql('Team', con=nba.engine, if_exists='append', index=False)  # Use the engine here

# Re-enable FK constraints and set IDENTITY_INSERT to OFF
with nba.engine.connect() as connection:
    # connection.execute(text("SET IDENTITY_INSERT Player OFF"))
    connection.execute(text("EXEC sp_MSforeachtable @command1='ALTER TABLE ? WITH CHECK CHECK CONSTRAINT ALL'"))


2023-08-23 01:13:26,856 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-08-23 01:13:26,857 INFO sqlalchemy.engine.Engine EXEC sp_MSforeachtable @command1='ALTER TABLE ? NOCHECK CONSTRAINT ALL'
2023-08-23 01:13:26,857 INFO sqlalchemy.engine.Engine [cached since 8.852e+04s ago] ()
2023-08-23 01:13:26,869 INFO sqlalchemy.engine.Engine ROLLBACK
2023-08-23 01:13:26,875 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-08-23 01:13:26,876 INFO sqlalchemy.engine.Engine SELECT [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] 
FROM [INFORMATION_SCHEMA].[TABLES] 
WHERE ([INFORMATION_SCHEMA].[TABLES].[TABLE_TYPE] = CAST(? AS NVARCHAR(max)) OR [INFORMATION_SCHEMA].[TABLES].[TABLE_TYPE] = CAST(? AS NVARCHAR(max))) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] = CAST(? AS NVARCHAR(max)) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_SCHEMA] = CAST(? AS NVARCHAR(max))
2023-08-23 01:13:26,877 INFO sqlalchemy.engine.Engine [cached since 8.865e+04s ago] ('BASE TABLE', 'VIEW', 'Team', 'dbo')
2023-08-23 01:13:2

#### Merge and Insert Games Played

In [243]:
# Query the database for teams and arena
with nba.engine.connect() as connection:
    teams = connection.execute(text("SELECT TeamID, TeamName FROM Team")).fetchall()
    teams = pd.DataFrame(teams, columns=['TeamID', 'TeamName'])
    arena = connection.execute(text("SELECT ArenaID, ArenaName FROM Arena")).fetchall()
    arena = pd.DataFrame(arena, columns=['ArenaID', 'ArenaName'])

2023-08-23 22:04:37,934 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-08-23 22:04:37,934 INFO sqlalchemy.engine.Engine SELECT TeamID, TeamName FROM Team
2023-08-23 22:04:37,935 INFO sqlalchemy.engine.Engine [cached since 7.488e+04s ago] ()
2023-08-23 22:04:37,946 INFO sqlalchemy.engine.Engine SELECT ArenaID, ArenaName FROM Arena
2023-08-23 22:04:37,946 INFO sqlalchemy.engine.Engine [cached since 7.488e+04s ago] ()
2023-08-23 22:04:37,949 INFO sqlalchemy.engine.Engine ROLLBACK


In [262]:
# Merge teams and arena with games_played
games_played = pd.read_csv(r"D:\lianz\Desktop\Python\personal_projects\nba_airflow\data\games_played.csv", parse_dates=['DateTime'])
games_played = games_played.merge(teams, left_on='Home', right_on='TeamName', how='left', suffixes=('_x', 'Home'), )
games_played = games_played.merge(teams, left_on='Visitor', right_on='TeamName', how='left', suffixes=('Home', 'Visitor'), )
games_played = games_played.merge(arena, left_on='Arena', right_on='ArenaName', how='left', suffixes=('_x', ''))
games_played = games_played[['DateTime', 'Visitor PTS', 'Home PTS', 'OT','Attendance','game_id', 'TeamIDHome','TeamIDVisitor', 'ArenaID']]
games_played.rename(columns={'Visitor PTS': 'VisitorPTS', 'Home PTS': 'HomePTS', 'TeamIDHome': 'HomeTeamID', 'TeamIDVisitor': 'VisitorTeamID', 'game_id': 'GameID'}, inplace=True)

In [265]:
# Disable FK constrains and set IDENTITY_INSERT to ON
with nba.engine.connect() as connection:
    connection.execute(text("EXEC sp_MSforeachtable @command1='ALTER TABLE ? NOCHECK CONSTRAINT ALL'"))

# Insert positions table
games_played.to_sql('GamesPlayed', con=nba.engine, if_exists='append', index=False)  # Use the engine here

# Re-enable FK constraints and set IDENTITY_INSERT to OFF
with nba.engine.connect() as connection:
    connection.execute(text("EXEC sp_MSforeachtable @command1='ALTER TABLE ? WITH CHECK CHECK CONSTRAINT ALL'"))

2023-08-23 22:14:28,856 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-08-23 22:14:28,857 INFO sqlalchemy.engine.Engine EXEC sp_MSforeachtable @command1='ALTER TABLE ? NOCHECK CONSTRAINT ALL'
2023-08-23 22:14:28,858 INFO sqlalchemy.engine.Engine [cached since 1.642e+05s ago] ()
2023-08-23 22:14:28,868 INFO sqlalchemy.engine.Engine ROLLBACK
2023-08-23 22:14:28,876 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-08-23 22:14:28,877 INFO sqlalchemy.engine.Engine SELECT [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] 
FROM [INFORMATION_SCHEMA].[TABLES] 
WHERE ([INFORMATION_SCHEMA].[TABLES].[TABLE_TYPE] = CAST(? AS NVARCHAR(max)) OR [INFORMATION_SCHEMA].[TABLES].[TABLE_TYPE] = CAST(? AS NVARCHAR(max))) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] = CAST(? AS NVARCHAR(max)) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_SCHEMA] = CAST(? AS NVARCHAR(max))
2023-08-23 22:14:28,877 INFO sqlalchemy.engine.Engine [cached since 1.643e+05s ago] ('BASE TABLE', 'VIEW', 'GamesPlayed', 'dbo')
2023-08-23 

#### Process play_by_play and make team_name based on full team name

In [ ]:
nba = DB(db_name='NBA')

sql_query_games = text("""
SELECT GameID, HomeTeamID, VisitorTeamID, HomeTeam.TeamName, VisitorTeam.TeamName FROM GamesPlayed
INNER JOIN Team AS HomeTeam ON GamesPlayed.HomeTeamID = HomeTeam.TeamID
INNER JOIN Team AS VisitorTeam ON GamesPlayed.VisitorTeamID = VisitorTeam.TeamID""")

sql_query_players = text("""
SELECT PlayerID, PlayerName, FromYear, ToYear FROM Player
WHERE ToYear >= 2000""")


with nba.engine.connect() as conn:
    games_played = conn.execute(sql_query_games).fetchall()
    games_played = pd.DataFrame(games_played, columns=['GameID', 'HomeTeamID', 'VisitorTeamID', 'HomeTeamName', 'VisitorTeamName'])
    
    players = conn.execute(sql_query_players).fetchall()
    players = pd.DataFrame(players, columns=['PlayerID', 'PlayerName', 'FromYear', 'ToYear'])

# Inner join games_played with play_by_play on gameid
pbp_final = games_played.merge(play_by_play, left_on='GameID', right_on='game_id', how='inner').drop('game_id', axis=1)
# Create new column for game year
pbp_final['game_year'] = pbp_final['datetime'].dt.year

def get_full_team_name(row):
    # Handle special cases first
    if row['team_name'].lower().strip() == "la clippers":
        return 23
    elif row['team_name'].lower().strip() == "la lakers":
        return 20

    z = re.match('.*{}.*'.format(row['team_name'].lower().strip()), row['VisitorTeamName'].lower().strip())
    x = re.match('.*{}.*'.format(row['team_name'].lower().strip()), row['HomeTeamName'].lower().strip())
    if z:
        # print(z.group().title())
        return row['VisitorTeamID']
    elif x:
        # print(x.group().title())
        return row['HomeTeamID']
    else:
        return None


pbp_final['TeamID'] = pbp_final.apply(get_full_team_name, axis=1)

# Merge dataframe based on PlayerName to get PlayerID as FK to Player table
pbp_final = pbp_final.merge(players, left_on='player_name', right_on='PlayerName', how='inner')

# Eliminate rows where game year is not between the player's FromYear and ToYear
pbp_final = pbp_final[(pbp_final['game_year'] >= pbp_final['FromYear']) & (pbp_final['game_year'] <= pbp_final['ToYear'])]

# Drop unnecessary columns
pbp_final.drop([
    'HomeTeamID', 
    'VisitorTeamID', 
    'HomeTeamName',
    'VisitorTeamName',
    'player_name',
    'team_name',
    'game_year', 
    'PlayerName', 
    'FromYear', 
    'ToYear'], axis=1, inplace=True)

# Rename Columns
pbp_final.rename(columns={
    'time_left': 'TimeLeft',
    'score_status': 'ScoreStatus',
    'x_shot_pos': 'X_Shot_Pos',
    'y_shot_pos': 'Y_Shot_Pos',
    'quarter': 'Quarter',
    'shot_status': 'ShotStatus',
    'full_text': 'FullText',
    'datetime': 'GameDate',}, inplace=True)

def generate_hash(row):
    # Convert each column to a string and concatenate to create a unique string
    unique_str = (
        str(row['PlayerID']) +
        str(row['TimeLeft']) +
        str(row['TeamID']) +
        str(row['ScoreStatus']) +
        str(row['X_Shot_Pos']) +
        str(row['Y_Shot_Pos']) +
        str(row['Quarter']) +
        str(row['ShotStatus']) +
        str(row['FullText']) +
        str(row['GameDate']) +
        str(row['GameID'])
    )
    
    # Hash the unique string using SHA-1 algorithm
    result = hashlib.sha1(unique_str.encode()).hexdigest()
    return result

# Create ShotsHashID column
pbp_final['ShotsHashID'] = pbp_final.apply(generate_hash, axis=1)
pbp_final.drop_duplicates(subset=['ShotsHashID'], inplace=True)

with nba.engine.connect() as connection:
    db_hashid = connection.execute(text("SELECT ShotsHashID FROM ShotsTaken")).fetchall()

# Check if hashid already in db
pbp_final = pbp_final.loc[~pbp_final['ShotsHashID'].isin([i[0] for i in db_hashid])].copy()

pbp_final.to_sql('ShotsTaken', con=nba.engine, if_exists='append', index=False)

In [307]:
test = pd.read_sql(text("SELECT TOP 1000 * FROM ShotsTaken"), con=nba.engine.connect())

def calculate_distance(x_shot: int, y_shot: int, x_basket: int, y_basket: int):
    x = x_shot - x_basket
    y = y_shot - y_basket
    d = np.sqrt(x**2 + y**2) / 10
    return round(d)

distance = np.vectorize(calculate_distance)
test['distance'] = distance(
    test['X_Shot_Pos'], test['Y_Shot_Pos'] * 1, 250, 417.5)
test['ft'] = test['FullText'].str.extract(r'(\d{1,2}) ft')


2023-08-26 00:35:51,967 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-08-26 00:35:51,968 INFO sqlalchemy.engine.Engine SELECT TOP 1000 * FROM ShotsTaken
2023-08-26 00:35:51,968 INFO sqlalchemy.engine.Engine [generated in 0.00110s] ()


In [309]:
for i in test[['X_Shot_Pos', 'Y_Shot_Pos', 'distance', 'ft']].values:
    print(i)

[83 268 22 '22']
[240 420 1 '0']
[240 420 1 '0']
[240 420 1 '0']
[258 373 5 '5']
[296 394 5 '6']
[240 420 1 '0']
[240 420 1 '0']
[240 420 1 '0']
[175 169 26 '26']
[296 118 30 '31']
[348 306 15 '16']
[240 420 1 '0']
[240 420 1 '0']
[240 420 1 '0']
[240 420 1 '0']
[5 291 28 '27']
[240 420 1 '0']
[240 420 1 '0']
[334 268 17 '18']
[198 207 22 '22']
[240 420 1 '0']
[240 420 1 '0']
[240 420 1 '0']
[270 163 26 '26']
[240 420 1 '0']
[393 207 25 '26']
[213 220 20 '20']
[363 279 18 '19']
[240 420 1 '0']
[240 420 1 '0']
[207 340 9 '9']
[240 420 1 '0']
[240 420 1 '0']
[240 420 1 '0']
[240 420 1 '0']
[240 420 1 '0']
[241 308 11 '11']
[240 420 1 '0']
[240 420 1 '0']
[240 420 1 '0']
[240 420 1 '0']
[241 253 16 '17']
[240 420 1 '0']
[240 420 1 '0']
[428 236 25 '26']
[236 357 6 '6']
[240 420 1 '0']
[393 220 24 '25']
[344 405 9 '11']
[240 420 1 '0']
[68 220 27 '26']
[240 420 1 '0']
[139 186 26 '25']
[116 333 16 '15']
[386 213 25 '25']
[232 285 13 '14']
[240 420 1 '0']
[240 420 1 '0']
[5 295 27 '27']
[24

##### Insert into SQL

In [220]:
nba = DB(db_name='NBA')
tempdb = DB(db_name='tempdb')

query_shotstaken = """
SELECT * FROM dbo.ShotsTaken
"""

create_temp_table_sql = """
CREATE TABLE #tempshots (
    ShotID INT,
    PlayerID INT,
    TimeLeft NVARCHAR(255),
    TeamID INT,
    ScoreStatus NVARCHAR(4),
    X_Shot_Pos INT,
    Y_Shot_Pos INT,
    Quarter NVARCHAR(255),
    ShotStatus NVARCHAR(255),
    FullText NVARCHAR(MAX),
    GameDate DATETIME,
    GameID NVARCHAR(30),
    ShotsHashID NVARCHAR(40)
);
"""
drop_temp_table_sql = """
DROP TABLE #tempshots;
"""

get_temptable = """
SELECT * FROM sys.tables
"""

query_temptable = """
SELECT * FROM #tempshots
"""


# Use MERGE to insert data from the temporary table into ShotsTaken
merge_sql = """
MERGE INTO ShotsTaken AS Target
USING #tempshots AS Source
ON Target.ShotsHashID = Source.ShotsHashID
WHEN NOT MATCHED BY TARGET THEN
    INSERT (PlayerID, TimeLeft, TeamID, ScoreStatus, X_Shot_Pos, Y_Shot_Pos, Quarter, ShotStatus, FullText, GameDate, GameID, ShotsHashID)
    VALUES (Source.PlayerID, Source.TimeLeft, Source.TeamID, Source.ScoreStatus, Source.X_Shot_Pos, Source.Y_Shot_Pos, Source.Quarter, Source.ShotStatus, Source.FullText, Source.GameDate, Source.GameID, Source.ShotsHashID);
"""


## Get positions

In [39]:
players = pd.read_csv(r'D:\lianz\Desktop\Python\personal_projects\nba_airflow\data\players.csv')

In [40]:
position = pd.DataFrame(players['Pos'].unique(), columns=['PositionName'])
descriptions = ['Forward-center', 'Center-forward', 'Center', 'Guard', 'Forward', 'Guard-forward', 'Forward-guard']
position['Description'] = descriptions

In [41]:
# Reset index to start at 1 and name it PositionID
position.index = position.index + 1
position.index.name = 'PositionID'
position.reset_index(inplace=True)

In [47]:
position.to_csv(r'D:\lianz\Desktop\Python\personal_projects\nba_airflow\data\position.csv', index=False)

## Create Table

### Columns to Create

In [5]:
player_shotLocations = pd.read_csv(r"D:\lianz\Desktop\Python\personal_projects\nba_airflow\data\player_shotLocations.csv", skipinitialspace=True, index_col=0)
games_played = pd.read_csv(r"D:\lianz\Desktop\Python\personal_projects\nba_airflow\data\games_played.csv", parse_dates=['DateTime'])
players = pd.read_csv(r"D:\lianz\Desktop\Python\personal_projects\nba_airflow\data\players.csv", parse_dates=['Birth Date'])
play_by_play = pd.read_parquet(r"D:\lianz\Desktop\Python\personal_projects\nba_airflow\data\play_by_play.parquet")

### Execute SQL to Create Table

In [ ]:
create_team_table = text("""
CREATE TABLE teams(
team_id INT PRIMARY KEY NOT NULL,
team_name VARCHAR(20),
team_abbreviation CHAR(3)
)
""")

create_player_table = text("""
CREATE TABLE players(
player_id INT PRIMARY KEY,
player_name VARCHAR(30),
from INT,
to INT,
position VARCHAR(5),
height VARCHAR(5),
weight INT,
birth_date datetime,
college VARCHAR(255),
) 
""")

create_team_arena = text("""
CREATE TABLE arena_history(
arena_id INT PRIMARY KEY,
arena_name VARCHAR(30),
team_year INT,
team_id INT FOREIGN KEY REFERENCES teams(team_id)
)
""")

create_player_team_table = text("""
CREATE TABLE player_team_history(
id INT NOT NULL IDENTITY(1,1),
player_id INT FOREIGN KEY REFERENCES players(player_id),
player_age INT,
team_id INT FOREIGN KEY REFERENCES teams(team_id),
year VARCHAR(10)
)
""")

create_games_played = text("""
CREATE TABLE games_played(
game_id CHAR(18) PRIMARY KEY,
game_date datetime,
visitor_team_id INT FOREIGN KEY REFERENCES teams(team_id), 
visitor_pts INT NOT NULL,
home_team_id INT FOREIGN KEY REFERENCES teams(team_id),
home_pts INT NOT NULL,
overtime VARCHAR(5),
attendance INT,
arena_id INT
)
""")

with nba.engine.begin() as conn:
    conn.execute(create_team_table)
    conn.execute(create_player_table)
    conn.execute(create_team_arena)
    conn.execute(create_player_team_table)
    conn.execute(create_games_played)

## Add Columns to Table

In [ ]:
add_arena_col = text("""
ALTER TABLE teams
ADD arena_name VARCHAR(30);
""")

with nba.engine.begin() as conn:
    conn.execute(add_arena_col)

## Insert Table

In [ ]:
games_played = games_played.dropna(subset=['Arena'])
games_played['DateStr'].apply(lambda x: len(str(x)))

In [ ]:
from sqlalchemy import DateTime, INT, VARCHAR

games_played.set_index('game_id').to_sql(name='games_played',
                    con=nba.engine,
                    if_exists='append',
                    dtype={'DateTime': DateTime,
                           'Visitor': VARCHAR(50),
                           'Visitor PTS': INT,
                           'Home': VARCHAR(50),
                           'Home PTS': INT,
                           'OT': VARCHAR(10),
                           'Attendance': INT, 
                           'Arena': VARCHAR(50), 
                           'DateStr': VARCHAR(12),
                           'Visitor_short': VARCHAR(3), 
                           'Home_short': VARCHAR(3),
                           'game_id': VARCHAR(18)})

In [ ]:
teams_df = games_played.rename(columns={'Home':'team_name', 'Home_short':'team_abbreviation',})[['DateTime','team_name','team_abbreviation','Arena']].copy()
teams_df['year'] = teams_df['DateTime'].apply(lambda x: str(x.year))
teams_df.drop(columns=['DateTime'], inplace=True)
teams_df = teams_df.dropna(subset=['Arena']).drop_duplicates()
teams_df['team_id'] = teams_df['team_abbreviation'] + teams_df['year']

### Execute SQL Statement to INSERT

In [ ]:
player_shotLocations[['PLAYER_ID', 'PLAYER_NAME','NICKNAME']].drop_duplicates('PLAYER_ID').to_sql('players',nba.engine, index=False)

In [ ]:
insert_table = text("""
INSERT INTO teams (team_id, team_name)
VALUES (:player_id, :player_name)
""")

with nba.engine.begin() as conn:
    conn.execute(insert_table, [{"player_id": 1, "player_name": 1}, {"player_id": 2, "player_name": 4}],)

# Drop Table

In [45]:
# Create function to drop table, can take in a list of tables
def drop_table(table_list):
    for table in table_list:
        table.drop(engine)
        print(f"Table {table} dropped")



In [ ]:
drop_table = text("""
DROP TABLE player_team_history;
DROP TABLE games_played;
DROP TABLE arena_history;
DROP TABLE players;
DROP TABLE teams;

""")

with nba.engine.begin() as conn:
    conn.execute(drop_table)

# Drop Column from Table

In [ ]:
column_to_drop = '[index]'
table_name = 'play_by_play'
drop_column = text(f"""
ALTER TABLE {table_name}
DROP COLUMN {column_to_drop}
""")

with nba.engine.begin() as conn:
    conn.execute(drop_column)
    print(f'Column {column_to_drop} has been dropped from table {table_name}')

# Access Data from Table

In [4]:
with nba.engine.connect() as conn:
    pbp = conn.execute(text("SELECT DISTINCT Player.PlayerName FROM ShotsTaken LEFT JOIN Player ON ShotsTaken.PlayerID = Player.PlayerID"))
    rows = pbp.fetchall()

pbp_df = pd.DataFrame(rows, columns=pbp.keys(),)

In [5]:
pbp_df

,PlayerName
0,Aaron Gray
1,Aaron McKie
2,Alfonzo McKinnie
3,Andray Blatche
4,Armon Johnson
...,...
2089,Wenyen Gabriel
2090,Wes Iwundu
2091,Willie Green
2092,Willy Hernangómez


In [314]:
special_names = [re.findall(r"[^a-z^A-Z^\s^'^-]", i) for i in pbp_df['PlayerName'] if re.findall(r"[^a-z^A-Z^\s^'^-]", i)]

In [317]:
chars = []

for i in special_names:
    chars.extend(i)

"".join(list(set(chars)))

'ÓèãýöŠáÁúŽêßäïšçí.òüôéžóë'

In [ ]:
pbp_df['player_name'].apply(lambda x: len(x)).max()

In [323]:
# Find all players who are not in the ShotsTaken table
with nba.engine.connect() as conn:
    players = conn.execute(text("SELECT PlayerName, FromYear, ToYear FROM Player"))
    players = players.fetchall()
    players = pd.DataFrame(players, columns=['PlayerName','FromYear','ToYear'])

with nba.engine.connect() as conn:
    pbp = conn.execute(text("SELECT DISTINCT Player.PlayerName FROM ShotsTaken LEFT JOIN Player ON ShotsTaken.PlayerID = Player.PlayerID"))
    rows = pbp.fetchall()
    pbp_df = pd.DataFrame(rows, columns=pbp.keys(),)

players.loc[(~players['PlayerName'].isin(pbp_df['PlayerName'])) & (players['ToYear'] >= 2000)]


2023-08-27 14:03:55,130 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-08-27 14:03:55,131 INFO sqlalchemy.engine.Engine SELECT PlayerName, FromYear, ToYear FROM Player
2023-08-27 14:03:55,132 INFO sqlalchemy.engine.Engine [cached since 28.38s ago] ()
2023-08-27 14:03:55,150 INFO sqlalchemy.engine.Engine ROLLBACK
2023-08-27 14:03:55,152 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-08-27 14:03:55,152 INFO sqlalchemy.engine.Engine SELECT DISTINCT Player.PlayerName FROM ShotsTaken LEFT JOIN Player ON ShotsTaken.PlayerID = Player.PlayerID
2023-08-27 14:03:55,152 INFO sqlalchemy.engine.Engine [cached since 629.4s ago] ()
2023-08-27 14:03:55,562 INFO sqlalchemy.engine.Engine ROLLBACK


,PlayerName,FromYear,ToYear
39,DeVaughn Akoon-Purcell,2019,2019
48,Cliff Alexander,2016,2016
69,Ray Allen*,1997,2014
112,Martynas Andriuškevicius,2006,2006
116,Chris Anstey,1998,2000
...,...,...,...
4976,Dedric Willoughby,2000,2000
4984,Jamil Wilson,2018,2018
5036,Haywoode Workman,1990,2000
5084,Tim Young,2000,2000


In [36]:
with nba.engine.connect() as connection:
    # Query for players
    players = connection.execute(text("SELECT PlayerID, PlayerName FROM Player"))
    column_names = players.keys()
    players_dict = [{col: getattr(p, col) for col in column_names} for p in players.fetchall()]
    # players = pd.DataFrame(players.fetchall(), columns=['PlayerID', 'PlayerName'])
# players.loc[players['PlayerName'].str.contains('Nikola')]

In [37]:
players_dict

[{'PlayerID': 1, 'PlayerName': 'Alaa Abdelnaby'},
 {'PlayerID': 2, 'PlayerName': 'Zaid Abdul-Aziz'},
 {'PlayerID': 3, 'PlayerName': 'Kareem Abdul-Jabbar*'},
 {'PlayerID': 4, 'PlayerName': 'Mahmoud Abdul-Rauf'},
 {'PlayerID': 5, 'PlayerName': 'Tariq Abdul-Wahad'},
 {'PlayerID': 6, 'PlayerName': 'Shareef Abdur-Rahim'},
 {'PlayerID': 7, 'PlayerName': 'Tom Abernethy'},
 {'PlayerID': 8, 'PlayerName': 'Forest Able'},
 {'PlayerID': 9, 'PlayerName': 'John Abramovic'},
 {'PlayerID': 10, 'PlayerName': 'Álex Abrines'},
 {'PlayerID': 11, 'PlayerName': 'Precious Achiuwa'},
 {'PlayerID': 12, 'PlayerName': 'Alex Acker'},
 {'PlayerID': 13, 'PlayerName': 'Don Ackerman'},
 {'PlayerID': 14, 'PlayerName': 'Mark Acres'},
 {'PlayerID': 15, 'PlayerName': 'Bud Acton'},
 {'PlayerID': 16, 'PlayerName': 'Quincy Acy'},
 {'PlayerID': 17, 'PlayerName': 'Alvan Adams'},
 {'PlayerID': 18, 'PlayerName': 'Don Adams'},
 {'PlayerID': 19, 'PlayerName': 'George Adams'},
 {'PlayerID': 20, 'PlayerName': 'Hassan Adams'},
 {'Pl

# Testing API Endpoint

In [6]:
import requests

tables = 'http://127.0.0.1:5000/api/shots?player_name=lebron&limit=100'
r = requests.get(tables)
print(r.json()['shots'])

[{'ShotID': 102129472, 'PlayerID': 2196, 'TimeLeft': '10:52.0', 'TeamID': 17, 'ScoreStatus': 'Cleveland trails 0-2', 'X_Shot_Pos': 224, 'Y_Shot_Pos': 231, 'Quarter': '1st quarter', 'ShotStatus': 'miss', 'FullText': '1st quarter, 10:52.0 remaining<br>LeBron James missed 2-pointer from 19 ft<br>Cleveland trails 0-2', 'GameDate': '2004-01-02T19:30:00', 'GameID': '200401021930CLENJN', 'ShotsHashID': 'a2463875f050476229f3cc37f4d9a2d34c86b3b7', 'PlayerName': 'LeBron James', 'PositionName': 'F-G', 'Height': '6-9', 'Weight': 250, 'BirthDate': '1984-12-30T00:00:00', 'College': None}, {'ShotID': 102129473, 'PlayerID': 2196, 'TimeLeft': '7:17.0', 'TeamID': 17, 'ScoreStatus': 'Cleveland leads 9-8', 'X_Shot_Pos': 138, 'Y_Shot_Pos': 329, 'Quarter': '1st quarter', 'ShotStatus': 'miss', 'FullText': '1st quarter, 7:17.0 remaining<br>LeBron James missed 2-pointer from 14 ft<br>Cleveland leads 9-8', 'GameDate': '2004-01-02T19:30:00', 'GameID': '200401021930CLENJN', 'ShotsHashID': '5606b0c82537d64133567eb

In [51]:
player_id = 2196
shots = 'http://127.0.0.1:5000/api/shots?player_id={}'.format(player_id)
r = requests.get(shots).json()['shots']
pd.DataFrame(r)

,ShotID,PlayerID,TimeLeft,TeamID,ScoreStatus,X_Shot_Pos,Y_Shot_Pos,Quarter,ShotStatus,FullText,GameDate,GameID,ShotsHashID,PlayerName,PositionName,Height,Weight,BirthDate,College
0,103159686,2107,0:01.9,1,Atlanta trails 33-71,240,420,2nd quarter,miss,"2nd quarter, 0:01.9 remaining<br>Othello Hunte...",2009-01-09T19:00:00,200901091900ATLORL,1917c89060575a2956c1e1e83599ce800ae7d1aa,Othello Hunter,F,6-8,225,1986-05-28T00:00:00,Ohio State
1,103159687,2107,0:27.3,1,Atlanta now leads 97-80,240,420,4th quarter,make,"4th quarter, 0:27.3 remaining<br>Othello Hunte...",2009-01-14T22:30:00,200901142230ATLLAC,5b6ff9524a89fb4f721bd7dd6cb57b30afccb323,Othello Hunter,F,6-8,225,1986-05-28T00:00:00,Ohio State
2,103159688,2107,3:39.0,1,Atlanta leads 112-80,403,359,4th quarter,miss,"4th quarter, 3:39.0 remaining<br>Othello Hunte...",2009-01-23T19:30:00,200901231930MILATL,57c11efafd960c3c4b48aa7a187458a9e7ad6eab,Othello Hunter,F,6-8,225,1986-05-28T00:00:00,Ohio State
3,103159689,2107,0:01.9,1,Atlanta leads 57-55,405,301,2nd quarter,miss,"2nd quarter, 0:01.9 remaining<br>Othello Hunte...",2009-01-25T18:00:00,200901251800PHXATL,e4a5267bfb1412c8a68b9ebfa5a7d5815298c856,Othello Hunter,F,6-8,225,1986-05-28T00:00:00,Ohio State
4,103159690,2107,5:40.0,1,Atlanta trails 27-32,326,411,2nd quarter,miss,"2nd quarter, 5:40.0 remaining<br>Othello Hunte...",2009-04-14T19:00:00,200904141900MIAATL,9160b960fc02a54669d38b3c7cde6fbd18f86bcd,Othello Hunter,F,6-8,225,1986-05-28T00:00:00,Ohio State
5,103159691,2107,2:48.0,1,Atlanta tied 32-32,276,413,2nd quarter,miss,"2nd quarter, 2:48.0 remaining<br>Othello Hunte...",2009-04-14T19:00:00,200904141900MIAATL,304c5d31455fe5ccfc5579486f5710c9cb1f5d93,Othello Hunter,F,6-8,225,1986-05-28T00:00:00,Ohio State
6,103159692,2107,2:42.0,1,Atlanta now leads 34-32,240,420,2nd quarter,make,"2nd quarter, 2:42.0 remaining<br>Othello Hunte...",2009-04-14T19:00:00,200904141900MIAATL,fafd9ecb54d822ca928ca9f91b7be921f3520d2a,Othello Hunter,F,6-8,225,1986-05-28T00:00:00,Ohio State
7,103159693,2107,7:48.0,1,Atlanta now leads 71-64,266,408,4th quarter,make,"4th quarter, 7:48.0 remaining<br>Othello Hunte...",2009-04-14T19:00:00,200904141900MIAATL,fc64ff99f461b67e4f437f6a067abd4347e7a30b,Othello Hunter,F,6-8,225,1986-05-28T00:00:00,Ohio State
8,103159694,2107,6:27.0,1,Atlanta now leads 73-68,240,405,4th quarter,make,"4th quarter, 6:27.0 remaining<br>Othello Hunte...",2009-04-14T19:00:00,200904141900MIAATL,7f5ae552d0bd724f0547a1ce66f37cfebb3e6670,Othello Hunter,F,6-8,225,1986-05-28T00:00:00,Ohio State
9,103159695,2107,3:54.0,1,Atlanta now leads 78-71,240,420,4th quarter,make,"4th quarter, 3:54.0 remaining<br>Othello Hunte...",2009-04-14T19:00:00,200904141900MIAATL,156e043a22f54997f43f5955ba2a4a0597ddeeb6,Othello Hunter,F,6-8,225,1986-05-28T00:00:00,Ohio State


# List Ports

In [61]:
import psutil
import re 
def list_ports_and_processes():
    for conn in psutil.net_connections(kind='inet'):
        if (conn.status == 'LISTEN' or conn.status == 'ESTABLISHED') and re.match(".*python.*", psutil.Process(conn.pid).name()):
            try:
                pid = conn.pid
                process = psutil.Process(pid)
                print(f"Port: {conn.laddr.port}, Status: {conn.status}, Process: {process.name()}, PID: {pid}")
            except:
                continue
def kill_process_by_pid(pid):
    try:
        process = psutil.Process(pid)
        process.terminate()
        print(f"Terminated process with PID: {pid}")
    except psutil.NoSuchProcess as e:
        print(f"Process with PID {pid} not found: {e}")


In [ ]:
list_ports_and_processes()